<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-04 17:20:02  
@LastEditors: Liusen  
@LastEditTime: 2020-09-30 18:50:02   

In [ ]:
#bgr8  to jpe format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time

import threading
import inspect
import ctypes

##  Thread shutdown function

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

## Create two display windows, one for the original image and the other for the mask position

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
image_widget1 = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)
display(image_widget1)

In [ ]:
image = cv2.VideoCapture(0)

image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #set frame
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 62) 
image.set(cv2.CAP_PROP_CONTRAST, 63) 
image.set(cv2.CAP_PROP_EXPOSURE, 4800) 

ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(frame)

### The next part is divided into two parts, the first part is to test the appropriate HSV value.
### The second one is for color recognition by testing the appropriate HSV value.
### Because both processes use video streams, after running the first part, you need to close the corresponding thread, and then run the second part.

## Part I

In [ ]:
Hmin = widgets.FloatSlider(description='Hmin', min=0, max=180, step=1, orientation='Vertical')
Hmax = widgets.FloatSlider(description='Hmax', min=0, max=179, step=1, orientation='Vertical')
Smin = widgets.FloatSlider(description='Smin', min=0, max=255, step=1, orientation='Vertical')
Smax = widgets.FloatSlider(description='Smax', min=0, max=255, step=1, orientation='Vertical')
Vmin = widgets.FloatSlider(description='Vmin', min=0, max=255, step=1, orientation='Vertical')
Vmax = widgets.FloatSlider(description='Vmax', min=0, max=255, step=1, orientation='Vertical')
# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([Hmin,Hmax,Smin,Smax,Vmin,Vmax])
# display the container in this cell's output
display(slider_container)
import numpy as np

In [ ]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [ ]:
color = "red"
Hmin.value=color_hsv[color][0][0]
Smin.value=color_hsv[color][0][1]
Vmin.value=color_hsv[color][0][2]
Hmax.value=color_hsv[color][1][0]
Smax.value=color_hsv[color][1][1]
Vmax.value=color_hsv[color][1][2]

In [ ]:
def TEST():
    while True:
        ret, frame = image.read()
        color_lower = np.array([int(Hmin.value),int(Smin.value),int(Vmin.value)])
        color_upper = np.array([int(Hmax.value), int(Smax.value), int(Vmax.value)])
        #frame = cv2.resize(frame, (400, 400))
        frame_ = cv2.GaussianBlur(frame,(5,5),0)                    
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)  
        mask = cv2.erode(mask,None,iterations=2)
        mask = cv2.dilate(mask,None,iterations=2)
        mask = cv2.GaussianBlur(mask,(3,3),0)     
        cnts = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
        if 1: # Push button switch
            if len(cnts) > 0:
                cnt = max (cnts, key = cv2.contourArea)
                (color_x,color_y),color_radius = cv2.minEnclosingCircle(cnt)
                if color_radius > 10:
                # Mark the detected color with the original shape coil
                    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
                    # Proportion-Integration-Differentiation

        image_widget.value = bgr8_to_jpeg(frame)
        image_widget1.value = bgr8_to_jpeg(mask)
        # print(g_mode)

In [ ]:
thread1 = threading.Thread(target=TEST)
thread1.setDaemon(True)
thread1.start()

In [ ]:
stop_thread(thread1)

## Part II

## Create an array to store red HSV color area data

In [ ]:
global g_mode
g_mode = 0

In [ ]:
import numpy as np
global color_lower
color_lower = np.array([0, 43, 46])
global color_upper
color_upper = np.array([10, 255, 255])

In [ ]:
Redbutton = widgets.Button(     
    value=False,     
    description='Red',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
Greenbutton = widgets.Button(     
    value=False,     
    description='Green',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
Bluebutton = widgets.Button(     
    value=False,     
    description='Blue',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
Yellowbutton = widgets.Button(     
    value=False,     
    description='Yellow',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
Orangebutton = widgets.Button(     
    value=False,     
    description='Orange',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
Closebutton = widgets.Button(     
    value=False,     
    description='Close',     
    disabled=False,     
    button_style='', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )
output = widgets.Output()
display(Redbutton, Greenbutton, Bluebutton, Yellowbutton, Orangebutton, Closebutton, output)

def ALL_Uncheck():
    Redbutton.icon = 'uncheck'
    Greenbutton.icon = 'uncheck'
    Bluebutton.icon = 'uncheck'
    Yellowbutton.icon = 'uncheck'
    Orangebutton.icon = 'uncheck'
    
def on_Redbutton_clicked(b):
    global color_lower, color_upper, g_mode 
    global target_valuex, target_valuey
    ALL_Uncheck()
    b.icon = 'check'
    #color_lower = np.array([0, 43, 46])
    #color_upper = np.array([10, 255, 255])
    color_lower = np.array([0,43,89])
    color_upper = np.array([7, 255, 255])
    g_mode = 1
    with output:
        print("RedButton clicked.")

def on_Greenbutton_clicked(b):
    global color_lower, color_upper, g_mode 
    global target_valuex, target_valuey
    ALL_Uncheck()
    b.icon = 'check'
    #color_lower = np.array([35, 43, 46])
    #color_upper = np.array([77, 255, 255])
    color_lower = np.array([54,104,64])
    color_upper = np.array([78, 255, 255])
    g_mode = 1
    with output:
        print("GreenButton clicked.")
        
def on_Bluebutton_clicked(b):
    global color_lower, color_upper, g_mode 
    global target_valuex, target_valuey
    ALL_Uncheck()
    b.icon = 'check'
    #color_lower=np.array([100, 43, 46])
    #color_upper = np.array([124, 255, 255])
    color_lower = np.array([92,100,62])
    color_upper = np.array([121, 255, 255])
    g_mode = 1
    with output:
        print("Bluebutton clicked.")   

def on_Yellowbutton_clicked(b):
    global color_lower, color_upper, g_mode 
    global target_valuex, target_valuey
    ALL_Uncheck()
    b.icon = 'check'
    #color_lower = np.array([26, 43, 46])
    #color_upper = np.array([34, 255, 255])
    color_lower = np.array([26,100,91])
    color_upper = np.array([32, 255, 255])
    g_mode = 1
    with output:
        print("Yellowbutton clicked.")
        
def on_Orangebutton_clicked(b):
    global color_lower, color_upper, g_mode 
    global target_valuex, target_valuey
    ALL_Uncheck()
    b.icon = 'check'
    color_lower = np.array([11, 43, 46])
    color_upper = np.array([25, 255, 255])
    g_mode = 1
    with output:
        print("Orangebutton clicked.")
        
def on_Closebutton_clicked(b):  
    global g_mode 
    
    ALL_Uncheck()
    g_mode = 0
    with output:
        print("CloseButton clicked.")

Redbutton.on_click(on_Redbutton_clicked)
Greenbutton.on_click(on_Greenbutton_clicked)
Bluebutton.on_click(on_Bluebutton_clicked)
Yellowbutton.on_click(on_Yellowbutton_clicked)
Orangebutton.on_click(on_Orangebutton_clicked)
Closebutton.on_click(on_Closebutton_clicked)

##  Color recognition main function

In [ ]:
def Color_Recongnize():
    global color_lower, color_upper, g_mode 
    t_start = time.time()
    fps = 0
    while True:
        ret, frame = image.read()
        #frame = cv2.resize(frame, (400, 400))
        frame_ = cv2.GaussianBlur(frame,(5,5),0)                    
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)  
        mask = cv2.erode(mask,None,iterations=2)
        mask = cv2.dilate(mask,None,iterations=2)
        mask = cv2.GaussianBlur(mask,(3,3),0)     
        cnts = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
        if g_mode == 1: # Push button switch
            if len(cnts) > 0:
                cnt = max (cnts, key = cv2.contourArea)
                (color_x,color_y),color_radius = cv2.minEnclosingCircle(cnt)
                if color_radius > 10:
                # Mark the detected color with the original shape coil
                    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
                    # Proportion-Integration-Differentiation
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS " + str(int(mfps)), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)

        image_widget.value = bgr8_to_jpeg(frame)
        image_widget1.value = bgr8_to_jpeg(mask)
        # print(g_mode)


##  启动进程

In [ ]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

##  关闭进程

In [ ]:
stop_thread(thread1)


In [ ]:
image.release()              #Release the object after use, otherwise when the next program needs to use the object, it will be unusable because it is occupied